This Notebook's idea is to use correctly useful ML fuctions (like the other notebooks in this github). So All the result can be improved upon. 

Some of the new topics that we are going to deal here are:
+ RandomForestClassifier
+ KNNImputer 
+ accuracy_score

In [1]:
# import 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# This Libraries were use in the other example because we needed to predict a numerical value
# now we are predicting a category

# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_absolute_error

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# New libraries that we haven't use yet!
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# These functions are going to be use in the next notebook
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score



In [2]:
# This data you can find here: https://www.kaggle.com/competitions/spaceship-titanic/data
X_full = pd.read_csv('train.csv', index_col='PassengerId')
X_test_full = pd.read_csv('test.csv', index_col='PassengerId')

# Transported is the target, if there is no target eliminate row associated with it
column_to_predict = "Transported"
X_full.dropna(axis=0, subset=[column_to_predict], inplace=True)
y = X_full[column_to_predict]
X = X_full.copy()
X.drop([column_to_predict], axis=1, inplace=True)
X_test = X_test_full.copy()
X.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
PassengerId,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines


In [3]:
X.head()
all_columns = X.columns

In [4]:
def cleaning_data(X,delete_over=10,col_to_change_to_null=0.1):
    """ 
    X: Dataframe that has to be cleaned
    delete_over : columns that have more than delete_over categories will be deleted
    col_to_change_to_null : If a column has more than col_to_change_to_null*100% values in null 
    it will create a new column that will be name column_name + _is_null and would be a boolean
    marking 1 if it the value is null and 0 if not.
    
    """
    categorical_variables = [col for col in  X.columns if str(X[col].dtypes)=='object']
    #numerical_variables = [col for col in X.columns if str(X[col].dtypes)!='object']
    cardinalidad = {}
    for col in categorical_variables:
        cardinalidad[col] = len(list(X[col].unique()))
    # For now we delete categories with more values than..
    delete_over = delete_over
    columns_to_delete = [col for col in categorical_variables if len(list(X[col].unique()))>delete_over ]
    X.drop(columns=columns_to_delete,inplace = True, axis=1)
    
    # We are going to change columns with too many null.
    # We are not gonna delete them, will give them the chance to be important.
    # that means that having or not having the value is what is really important.
    col_to_change_to_null = col_to_change_to_null
    columnas_modificar_por_1 = [col for col in X.columns if X[col].isnull().sum()>int(X.shape[0] * col_to_change_to_null) ]

    for col in columnas_modificar_por_1:
        X[col +str('_is_null')] = 0
        X.loc[(X[col].isnull()),col +str('_is_null')] = 1

    new_columns_null = [str(f"{col}_is_null") for col in columnas_modificar_por_1 ]    
    X.drop(columns=columnas_modificar_por_1, axis=1,inplace=True)
    
    return X

In [5]:
X = cleaning_data(X)

In [6]:
new_columns = X.columns
X.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
PassengerId,,,,,,,,,,
0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0
0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0
0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0
0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0
0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0


In [7]:
# columns deleted given the criterion from the function cleaning_data
deleted_columns = set(all_columns) - set(new_columns)
deleted_columns

{'Cabin', 'Name'}

In [8]:
def transforming_modeling_scoring(X,y,cv=3,n_jobs=-1,n_iter=15, scoring='accuracy'):
    # separating the data in training/validation
    # X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
    #                                                  random_state=0)        
    
    # We are gonna use a cross validation now
    X_train = X
    y_train = y
    
    numerical_col = [col for col in X_train.columns if str(X_train[col].dtypes)!='object' ]
    numerical_col_imputed = [col for col in numerical_col if X_train[col].isnull().any()==True]

    categorical_col = [col for col in X_train.columns if str(X_train[col].dtypes)=='object' ]
    categorical_col_imputed = [col for col in categorical_col if X_train[col].isnull().any()==True]

    numerical_transformer = SimpleImputer(strategy='mean')

    categorical_transformer =  Pipeline(steps=[
        ('imputer', SimpleImputer(missing_values=pd.NA, strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformer(transformers=
        [("numerical_transformer", numerical_transformer, numerical_col_imputed),
        ("categorical_transformer", categorical_transformer, categorical_col)],remainder='passthrough')

    # Define model
    model = RandomForestClassifier(random_state=0,n_jobs=n_jobs)

    # Bundle preprocessing and modeling code in a pipeline
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                          ('model', model)
                         ])

    # pipe.fit(X_train,y_train)
    
    # scores = -1 * cross_val_score(pipe, X, y,cv=cv,scoring='accuracy')
    distributions = dict(model__n_estimators=randint(low=10,high=500))
    clf = RandomizedSearchCV(pipe, distributions, random_state=0, cv=cv,n_iter=n_iter,scoring=scoring)
    search = clf.fit(X_train,y_train)
    
    return search

In [9]:
search = transforming_modeling_scoring(X,y)

In [20]:
#search

In [11]:
search.best_params_

{'model__n_estimators': 97}

In [12]:
(search.best_score_)

0.7793640629482805

here we are creating a 2 version of the function above because we are going to try a new imputer! to see how it does. The new imputer is KNNImputer.

In [13]:
def transforming_modeling_scoring_2(X,y,cv=3,n_jobs=-1,n_iter=15, scoring='accuracy'):
    # separating the data in training/validation
    # X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
    #                                                  random_state=0)        
    
    # We are gonna use a cross validation now
    X_train = X
    y_train = y
    
    numerical_col = [col for col in X_train.columns if str(X_train[col].dtypes)!='object' ]
    numerical_col_imputed = [col for col in numerical_col if X_train[col].isnull().any()==True]

    categorical_col = [col for col in X_train.columns if str(X_train[col].dtypes)=='object' ]
    categorical_col_imputed = [col for col in categorical_col if X_train[col].isnull().any()==True]
    # n_neighbors = an integer. KNNImputer 
    numerical_transformer = Pipeline(
    steps=[("scaler", StandardScaler()), ("imputer", KNNImputer())
          ]
    )

    categorical_transformer =  Pipeline(steps=[
        ('imputer', SimpleImputer(missing_values=pd.NA, strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformer(transformers=
        [("numerical_transformer", numerical_transformer, numerical_col_imputed),
        ("categorical_transformer", categorical_transformer, categorical_col)],remainder='passthrough')

    # Define model
    model = RandomForestClassifier(random_state=0,n_jobs=n_jobs)

    # Bundle preprocessing and modeling code in a pipeline
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                          ('model', model)
                         ])

    # pipe.fit(X_train,y_train)
    
    # scores = -1 * cross_val_score(pipe, X, y,cv=cv,scoring='accuracy')
    distributions = dict(model__n_estimators=randint(low=10,high=500))
    
    distributions = {
    'preprocessor__numerical_transformer__imputer__n_neighbors':randint(low=1,high=7),
    'model__max_depth':randint(low=2,high=20),
    'model__n_estimators':randint(low=10,high=500),
      }
    clf = RandomizedSearchCV(pipe, distributions, random_state=0, cv=cv,n_iter=n_iter,scoring=scoring)
    search = clf.fit(X_train,y_train)
    
    return search

In [14]:
search = transforming_modeling_scoring_2(X,y)

In [15]:
search.best_params_

{'model__max_depth': 8,
 'model__n_estimators': 49,
 'preprocessor__numerical_transformer__imputer__n_neighbors': 1}

In [16]:
(search.best_score_)

0.7951240818599855

Notemos todo lo que mejoró nuestro prónostico permitiendo a nuestro modelo escoger los mejores parametros en base a accuracy. Note también que cambiamos el simple imputer por uno más elaborado (KNNImputer).

In [18]:
#search